# LLMs

Here we'll be interacting with a server that's exposing 2 LLMs via the runnable interface.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

In [ ]:
from langserve import RemoteRunnable

# vertexai = RemoteRunnable("http://localhost:8000/vertexai/")
vertexai = RemoteRunnable("https://streaming-piaidmycda-uc.a.run.app/vertexai/")

Let's create a prompt composed of a system message and a human message.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences.",
        ),
        ("human", "Tell me about your favorite novel"),
    ]
).format_messages()

### Not Streamed

In [ ]:
vertexai.invoke(prompt)

As with regular runnables, async invoke, batch and async batch variants are available by default

## Streamed

In [ ]:
for chunk in vertexai.stream(prompt):
    print(chunk.content, end="", flush=True)

In [ ]:
async for chunk in vertexai.astream(prompt):
    print(chunk.content, end="", flush=True)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | vertexai
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | vertexai
)


chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [ ]:
chain.invoke({})